06.09.16

## Cholesky Decomposition

Implement Cholesky-Crout algorithm to calculate the matrix decomposition in the form $$\mathbf{A} = \mathbf{L}\mathbf{L}^\mathrm{T},$$ where $\mathbf{A}$ is a symmetric positive-definite matrix and $\mathbf{L}$ is a lower-triangular matrix with real and positive diagonal entries.

Computational complexity: $\mathcal{O}(n^3)$. Twice as efficient compared to LU-decomposition: $2n^3/3$ vs $n^3/3$ flops).

In [23]:
# A = matrix(RDF, [[4, 12, -16],
#                 [12, 37, -43],
#                 [-16, -43, 98]])
A = matrix(RDF, [[1.25, 0.34, 0.01, -1.27],
                 [0.34, -1.68, -2.04, 0.25],
                 [0.01, -2.04, 0.94, 2.45],
                 [-1.27, 0.25, 2.45, 0.85]])
# b = vector(RDF, [2, 5, 1])
b = vector(RDF, [4.241942, -0.030203, -7.411342, -6.289385])
show('\mathbf{A}=' + latex(A))
show('\mathbf{b}=' + latex(b))

\mathbf{A}= \left(\begin{array}{rrrr}
1.25 & 0.34 & 0.01 & -1.27 \\
0.34 & -1.68 & -2.04 & 0.25 \\
0.01 & -2.04 & 0.94 & 2.45 \\
-1.27 & 0.25 & 2.45 & 0.85
\end{array}\right)

\mathbf{b}= \left(4.241942,\,-0.030203,\,-7.411342,\,-6.289385\right)

In [24]:
def cholesky_decomposition(A):
    L = zero_matrix(CC, A.nrows(), A.ncols())
    for i in range(A.nrows()):
        for j in range(i+1):
            if i == j:
                L[i, j] = sqrt(A[i, j] - sum(L[j,k]^2 for k in range(j)))
            else:
                L[i, j] = 1/L[j, j] * (A[i, j] - sum(L[i, k] * L[j, k] for k in range(j)))
    return L, L.T

In [25]:
L, U = cholesky_decomposition(A)

In [26]:
show('\mathbf{L}=' + latex(L))
show('\mathbf{U}=' + latex(U))
show('\mathbf{L}\mathbf{U}=' + latex(L*U))
show('\mathbf{A}=' + latex(A))
show('\mathbf{A} - \mathbf{LU}=' + latex(A - L*U))

\mathbf{L}= \left(\begin{array}{rrrr}
1.11803398874989 & 0.000000000000000 & 0.000000000000000 & 0.000000000000000 \\
0.304105244939971 & 1.33134518439058i & 0.000000000000000 & 0.000000000000000 \\
0.00894427190999916 & 1.53432785422591i & 1.81496059578534 & 0.000000000000000 \\
-1.13592253256989 & -0.447246895081203i & 0.977396773946638 & 1.09343251669297i
\end{array}\right)

\mathbf{U}= \left(\begin{array}{rrrr}
1.11803398874989 & 0.304105244939971 & 0.00894427190999916 & -1.13592253256989 \\
0.000000000000000 & 1.33134518439058i & 1.53432785422591i & -0.447246895081203i \\
0.000000000000000 & 0.000000000000000 & 1.81496059578534 & 0.977396773946638 \\
0.000000000000000 & 0.000000000000000 & 0.000000000000000 & 1.09343251669297i
\end{array}\right)

\mathbf{L}\mathbf{U}= \left(\begin{array}{rrrr}
1.25000000000000 & 0.340000000000000 & 0.0100000000000000 & -1.27000000000000 \\
0.340000000000000 & -1.68000000000000 & -2.04000000000000 & 0.250000000000000 \\
0.0100000000000000 & -2.04000000000000 & 0.940000000000000 & 2.45000000000000 \\
-1.27000000000000 & 0.250000000000000 & 2.45000000000000 & 0.850000000000000
\end{array}\right)

\mathbf{A}= \left(\begin{array}{rrrr}
1.25 & 0.34 & 0.01 & -1.27 \\
0.34 & -1.68 & -2.04 & 0.25 \\
0.01 & -2.04 & 0.94 & 2.45 \\
-1.27 & 0.25 & 2.45 & 0.85
\end{array}\right)

\mathbf{A} - \mathbf{LU}= \left(\begin{array}{rrrr}
-2.22044604925031 \times 10^{-16} & 5.55111512312578 \times 10^{-17} & 0.000000000000000 & -2.22044604925031 \times 10^{-16} \\
5.55111512312578 \times 10^{-17} & -2.22044604925031 \times 10^{-16} & 0.000000000000000 & -5.55111512312578 \times 10^{-17} \\
0.000000000000000 & 0.000000000000000 & 0.000000000000000 & 0.000000000000000 \\
-2.22044604925031 \times 10^{-16} & -5.55111512312578 \times 10^{-17} & 0.000000000000000 & 1.11022302462516 \times 10^{-16}
\end{array}\right)

## SLEQ solution via Cholesky Decomposition

Solve $$\mathbf{b}=\mathbf{A}\mathbf{x}=\mathbf{L}\underbrace{\mathbf{U}\mathbf{x}}_{\mathbf{y}}$$
  for $\mathbf{y}$
  to find $\mathbf{y}^{*}$
 ; then $\mathbf{U}\mathbf{x}=\mathbf{y}^{*}$
  to obtain the desired $\mathbf{x}^{*}$.

In [27]:
def substitution(A, b, kind='forward'):
    N = A.nrows()
    x_star = vector(CC, [0 for _ in range(N)])
    if kind == 'backward':
        for i in reversed(range(N)):
            x_star[i] = (b[i] - sum(A[i, k] * x_star[k] for k in range(i+1, N))) / A[i, i]
    elif kind == 'forward':
        for i in range(N):
            x_star[i] = (b[i] - sum(A[i, k] * x_star[k] for k in range(i))) / A[i, i]
    else:
        raise ValueError("Unknown substitution type: %s" % kind)
    return x_star

In [28]:
y_star = substitution(L, b, kind='forward')
show('\mathbf{y^*}=' + latex(y_star))

\mathbf{y^*}= \left(3.79410826744456,\,0.889334515106970i,\,-3.35034646585025,\,-0.820621103778071i\right)

In [29]:
x_star = substitution(U, y_star, kind='backward')
show('\mathbf{x^*}=' + latex(x_star))

\mathbf{x^*}= \left(2.07750000000000,\,2.07750000000000,\,-1.44180000000000,\,-0.750499999999998\right)

### Check the answer

In [30]:
show('\mathbf{A}\mathbf{x}^*=' + latex(A*x_star))
show('\mathbf{b}=' + latex(b))
show('\mathbf{A}\mathbf{x}^* - \mathbf{b}=' + latex(A*x_star - b))

\mathbf{A}\mathbf{x}^*= \left(4.24194200000000,\,-0.0302030000000014,\,-7.41134200000000,\,-6.28938500000000\right)

\mathbf{b}= \left(4.241942,\,-0.030203,\,-7.411342,\,-6.289385\right)

\mathbf{A}\mathbf{x}^* - \mathbf{b}= \left(-8.88178419700125 \times 10^{-16},\,-1.39471767468535 \times 10^{-15},\,0.000000000000000,\,0.000000000000000\right)